# Reading, writing and editing key-value pairs

Key value pairs are a key (pun intended) part of annotating data in OMERO in a very flexible way. In essence, they provide a way of adding Python dictionaries to an image, a project or a dataset - and a Python dictionary is fit for data of virtually any form. Here are a few ways of interacting with data in this form through Python.

In [29]:
import ezomero
from omero.rtypes import rstring, rlong

In [2]:
host = 'omero-int.biotec.tu-dresden.de'
user = 'johamuel'  # replace this with your username
secure = True
port = 4064
group = 'default'

conn = ezomero.connect(host=host, user=user, secure=secure, port=port, group=group)

## Adding key value pairs

Let's add some kv pairs from a Python dictionary. In the OMERO terminology, key value pairs are referred to as "map annotations", so that's the function we'll need to use:

In [3]:
kv_pairs = {'Age': 35, 'Patient name': 'John Doe'}

In [8]:
map_annotation_id = ezomero.post_map_annotation(
    conn,
    object_type='Image',
    object_id=330,
    kv_dict=kv_pairs,
    ns='openmicroscopy.org/omero/client/mapAnnotation')
print(map_annotation_id)

object group 3
8257


If you check out your image on OMERo, you see the posted key-value pairs:

![kv-pairs](./imgs/kv_pairs.PNG)

Let's quickly talk about the elephant the room: What exactly is the `ns` argument of the `post_map_annotation` function? It stands for "namespace" and is a way of grouping key-value pairs. If you have a lot of key-value pairs, you might want to group them into different namespaces to keep things organized. If you don't care about namespaces, you can just use an empty string. The default namespace is `'openmicroscopy.org/omero/client/mapAnnotation'`. 

```{note}
If you set the namespace to a value different from 'openmicroscopy.org/omero/client/mapAnnotation', you will not be able to edit the key-value pairs in the OMERO web interface.
```

## Reading a key-value pair from OMERO

Let's now do it the other way round: read a key-value pair from OMERO. For this, we need a two-step process: We first need to retrieve all map annotations of a single image (there can be more than one) and then we can access the key-value pairs of the individual annotations.

In [9]:
map_annotation_ids = ezomero.get_map_annotation_ids(conn, object_type='Image', object_id=330)
print(map_annotation_ids)a

[8257]


Let's look at the first (and only) map annotation we just retrieved:

In [11]:
kv_pairs = ezomero.get_map_annotation(conn, map_ann_id=map_annotation_ids[0])
kv_pairs

{'Age': '35', 'Patient name': 'John Doe'}

## Edit key-value pairs

Lastly, let's say we don't want to read or write a kv pairs, but we really just want to edit a particular entry of a key-value dictionary. We start just as before by getting the map_annotation_id of the particular map annotation we are interested in.

In [12]:
map_annotation_ids = ezomero.get_map_annotation_ids(conn, object_type='Image', object_id=330)
print(map_annotation_ids)

[8257]


Now we retrieve the map annotation as an omero object:

In [42]:
mapAnnotation = conn.getObject(obj_type='MapAnnotation', oid=map_annotation_ids[0])

This is the new dictoinary we want to save.

```{warning}
Setting a key-value pair like this will overwrite the entire map annotation. If you want to keep the existing key-value pairs, you will need to read the existing key-value pairs first and then update or append the dictionary.
```

In [45]:
kv_pairs = [
    ('Age', '36'),
    ('Patient name', 'John Doe2')
    ]  # Note: the value should be a string - no integers allowed here

In [46]:
# Save the updated annotation
mapAnnotation.setValue(kv_pairs)
conn.getUpdateService().saveObject(mapAnnotation._obj)

There's your result:

![kv-pairs](./imgs/kv_pairs_edited.PNG)